In [111]:
import numpy as np
import pandas as pd
from scipy.io import loadmat, mmread
import sys
import itertools 
import pickle
import matplotlib.pylab as plt
import time

In [112]:
%matplotlib inline

In [113]:
import rankingmethod
import rankingmeasure

In [114]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:
from experiment import *

# Epinion

In [ ]:
data = pd.read_csv("../datasets/soc-Epinions1.txt", sep='\t', header=4)
data = data.as_matrix()

In [ ]:
data.shape

In [ ]:
#создание epinion удалив от туда пользователей, у которых количество отзывов меньше 25
#Также проводится переиндексанция пользователей и предметов

In [ ]:
epinion, user_item = make_valid_data(data)

In [ ]:
train, test, traintest = givenK_train_test(epinion, 5)

### изучение числа предметов у пользователей

In [ ]:
avr_len = 0
max_len = 0
for items in traintest:
    avr_len += len(items)
    max_len = max(len(items), max_len)
avr_len /= len(traintest)
print(avr_len)
print(max_len)

In [ ]:
list_len = []
for items in traintest:
    list_len.append(len(items))
list_len = np.array(list_len)

In [ ]:
np.sum(list_len > 1200)

In [ ]:
plt.hist(list_len, bins = 30)

In [ ]:
number_user_in_items = np.bincount(epinion[:, 1])

In [ ]:
plt.hist(np.bincount(epinion[:, 1]))

### подбор параметров для отдельных алгоритмов

In [99]:
poprec = rankingmethod.PopRec(user_item)
poprec.fit(train)
top = poprec.get_list(0)[:3]
# all_measures(train, test, poprec, True, top)
all_measures(train, test, poprec, True)

precK 0.20113507378
1-call5 0.5743473325766175
MRR 0.4425874069367828
AUC 0.830356545287
NGDC 0.223636021272
MAP 0.384312965065


In [ ]:
###CLimf подбор параметров(не используя фичу авторов)

In [ ]:
list_reg = [0.001, 0.05, 0.01, 0.005 , 0.001]
list_lrate = [0.01, 0.0001, 0.005, 0.01, 0.05, 0.1 ]
list_K = [10, 5, 7, 12, 15]

climf_validation = np.zeros((len(list_reg), len(list_lrate), len(list_K)))

for i, K in enumerate(list_K):
    for j, lrate in enumerate(list_lrate):
        for m, reg in enumerate(list_reg):
            climf = rankingmethod.CLiMF(user_item,K=K, reg=reg, lrate=lrate, verbose=0, maxiter=10)
            climf.fit(train)
            mrr = rankingmeasure.get_MRR(train, test, 
            climf, skip_train=False)
            climf_validation[i,j,m] = mrr
            print(K, lrate, reg, "MRR", mrr)
            sys.stdout.flush()

###проверка работы авторского решения

In [ ]:
create_csr(train, user_item, 'another_train.mtx')
create_csr(test, user_item, 'another_test.mtx')

###BPR_MF Подбор параметров

In [ ]:
list_reg = [0.01, 0.05, 0.1, 0.2]
list_lrate = [0.0001, 0.001, 0.005]
list_K = [10, 15,  20]

bpr_mf_validation = np.zeros((len(list_K), len(list_lrate), len(list_reg)))

for i, K in enumerate(list_K):
    for j, lrate in enumerate(list_lrate):
        for m, reg in enumerate(list_reg):
            cf = rankingmethod.BPR_MF(user_item,K=K, regU=reg, regIpos=reg, regIneg=reg, 
                                      lrate=lrate, verbose=0, maxiter=30)
            cf.fit(train)
            res = rankingmeasure.get_AUC(train, test, cf)
            bpr_mf_validation[i,j,m] = res
            print(K, lrate, reg, "AUC", res)
            sys.stdout.flush()

In [ ]:
with open('bpr_mf_validation.pickle', 'wb') as f:
    f.dump(bpr_mf_validation, f)

###iMF Подбор  параметров

In [ ]:
list_alpha = [10, 20, 40, 80]
list_lmbd = [0.001, 0.01, 0.05, 0.1]
list_K = [10, 15,  20]

imf_validation = np.zeros((len(list_K), len(list_alpha), len(list_lmbd)))

for i, K in enumerate(list_K):
    for j, alpha in enumerate(list_alpha):
        for m, lmbd in enumerate(list_lmbd):
            cf = rankingmethod.iMF(user_item, K=K, lmbd=lmbd, alpha=alpha, maxiter=30, verbose=0)
            cf.fit(train)
            mrr = rankingmeasure.get_prec_K(train, test, cf)
            imf_validation[i,j,m] = mrr
            print(K, lrate, reg, "prec", mrr)
            sys.stdout.flush()

### TFMA

In [103]:
cf = rankingmethod.TFMAP(user_item, lrate=0.01, verbose = 1, maxiter=12)
cf.fit(train)

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11


In [102]:
all_measures(train, test, cf)

precK 0.180930760499
1-call5 0.5402951191827469
MRR 0.42529711390083835
AUC 0.630444505232
NGDC 0.207541948703
MAP 0.373795875899


In [105]:
user_item

array([ 4405, 34777])

### pureSVD

In [143]:
cf = rankingmethod.pureSVD(user_item,  rate_neg_sample=1000.0, verbose=1)


In [ ]:
cf.fit(train)

In [142]:
all_measures(train, test, cf)

precK 0.00690124858116
1-call5 0.027695800227014757
MRR 0.030906862455593657
AUC 0.806544835437
NGDC 0.0059649372233
MAP 0.0101226510279


In [139]:
all_measures(train, test, cf)

precK 0.00989784335982
1-call5 0.038138479001135074
MRR 0.03823556637728849
AUC 0.805579474585
NGDC 0.0104280871446
MAP 0.0212220961029


## Проведение такого же эксперимента, как у создатей CLiMF

In [ ]:
def climf_experiment(data, list_cf, list_measure, K, maxiter, top=None):
    res = np.zeros((len(list_measure), len(list_cf)))
    for t in range(maxiter):
        start_time = time.time()
        train, test, traintest = givenK_train_test(epinion, K)
        print('create test and train', time.time() - start_time)
        for i, cf in enumerate(list_cf):
            start_time = time.time()
            cf.fit(train)
            print('fit method', time.time() - start_time)
            for j, measure in enumerate(list_measure):
                start_time = time.time()
                res[j, i] += measure(train, test, cf, top=top)
                print('get measure', time.time() - start_time)
            print(t, i, res[:, i] / (t + 1))
            sys.stdout.flush()
    res /= K
    return res

In [ ]:
climf_experiment(epinion, list_cf, list_measure, 5, 1, None)

In [ ]:
list_cf = [
    rankingmethod.PopRec(user_item),
    rankingmethod.CLiMF(user_item, K=10, reg=0.001, lrate=0.01, verbose=0, maxiter=10),
    rankingmethod.BPR_MF(user_item, K = 20, lrate=0.001, regIpos=0.1, 
                              regIneg=0.1, regU=0.1, maxiter=30, verbose=0 ),
    rankingmethod.iMF(user_item, K=10, lmbd=0.01, alpha=40, maxiter=30, verbose=0)
    
]

list_measure = [
    rankingmeasure.get_prec_K,
    rankingmeasure.get_one_recal_K,
    rankingmeasure.get_MRR,
    rankingmeasure.get_AUC,
    rankingmeasure.get_NDCG,
    rankingmeasure.get_MAP
]

In [ ]:
res5 = climf_experiment(epinion, list_cf, list_measure, 5, 5, None)

In [ ]:
for i in range(5, 21, 5):
    res = climf_experiment(epinion, list_cf, list_measure, i, 5, None)
    with open('climf_res' + str(i) +'.pickle', 'wb') as f:
        pickle.dump(res, f)
    res = climf_experiment(epinion, list_cf, list_measure, i, 5, top)
    with open('climf_res_top' + str(i) + '.pickle', 'wb') as f:
        pickle.dump(res, f)

In [ ]:
def show_smart_table(data):
    name_measure = ['precK', '1-recalK', 'MRR', 'AUC', 'NDCG']
    ans = pd.DataFrame()
    ans['measures'] = name_measure
    ans['PopRec'] = data[:, 0]
    ans['CLiMF'] = data[:, 1]
    ans['BRP_MP'] = data[:, 2]
    ans['iMF'] = data[:, 3]
    print(ans)

In [ ]:
for i in range(5, 21, 5):
    with open('climf_res' + str(i) +'.pickle', 'rb') as f:
        tbl = pickle.load(f)
    print('---------')
    print('K =', i)
    show_smart_table(tbl)
    with open('climf_res_top' + str(i) + '.pickle', 'rb') as f:
        tbl = pickle.load(f)
    print('---------')
    print('K =', i, 'without top')
    show_smart_table(tbl)

In [ ]:
show_smart_table(res5)

# старые эксперименты

In [ ]:
a = np.arange(10)
b = a[1:4]
print(a)
print(b)
b[2] = 100
a[1] = -a[1]
print(a)
print(b)

In [ ]:
tmp = mmread("../datasets/epinions_original/EP25_UPL5_test.mtx")
test = np.array(list(zip(tmp.row, tmp.col)))
tmp = mmread("../datasets/epinions_original/EP25_UPL5_train.mtx")
train = np.array(list(zip(tmp.row, tmp.col)))
user_item = [max(np.max(test[:,0]), np.max(train[:, 0])) + 1, max(np.max(test[:,1]), np.max(train[:,1])) + 1]

In [ ]:
train = create_listarray(train)
test  = create_listarray(test)

In [ ]:
create_csr(train, user_item, '../pyCLiMF-master/good_train.pickle')
create_csr(test, user_item, '../pyCLiMF-master/good_test.pickle')

In [ ]:
randomrec = rankingmethod.RandomRec(user_item)
randomrec.fit(train)
all_measures(train, test, randomrec, False)

In [ ]:
poprec = rankingmethod.PopRec(user_item)
poprec.fit(train)
top = poprec.get_list(0)[:3]
# all_measures(train, test, poprec, True, top)
# all_measures(train, test, poprec, True)

In [ ]:
rankingmeasure.get_AUC(train, test, poprec, True, K=user_item[1])

In [ ]:
rankingmeasure.get_NDCG(train, test, poprec, True, K = 5)

In [ ]:
rankingmeasure.get_NDCG(train, test, poprec, True, K = 3)

In [ ]:
index_user = np.random.choice(len(train), 100, replace=False)


In [ ]:
orig_train = create_original_sample(train, index_user)
orig_test = create_original_sample(test, index_user)

In [ ]:
climf = rankingmethod.CLiMF(user_item, lrate = 0.0001, reg=0.001, maxiter=15, verbose=2)
climf.fit(train)

In [ ]:
rankingmeasure.get_MRR(train, test, climf, True)

In [ ]:
all_measures(train, test, climf, True, top)

In [ ]:
all_measures(train, test, climf, True)

In [ ]:
climf = rankingmethod.CLiMF(user_item, maxiter=40, lrate = 0.1, verbose=2)
climf.fit(train)


In [ ]:
all_measures(train, test, climf, False)

In [ ]:
all_measures(train, test, climf, True, top)

In [ ]:
list_reg = [0.001, 0.002, 0.005, 0.0005]
list_lrate = [0.05, 0.07, 0.1, 0.12, 0.15 ]
list_K = [10, 12, 15, 7, 5]
times_count = 3

for K, lrate, reg in itertools.product(list_K, list_lrate, list_reg):
    MRR = 0
    
    for t in range(times_count):
        climf = rankingmethod.CLiMF(user_item,K=K, reg=reg, lrate=lrate, verbose=0, maxiter=50)
        climf.fit(train)
        MRR += rankingmeasure.get_MRR(train, test, climf, skip_train=False)
    print("K", K, "lrate", lrate, "reg", reg, "MRR", MRR/times_count)
    sys.stdout.flush()

In [ ]:
bpr_mf = rankingmethod.BPR_MF(user_item, K = 20, lrate=0.001, regIpos=0.1, 
                              regIneg=0.1, regU=0.1, maxiter=30, verbose=2 )

In [ ]:
bpr_mf.fit(train)

In [ ]:
all_measures(train, test, bpr_mf, True)

In [ ]:
rankingmeasure.get_AUC(train, test, bpr_mf, True, K=user_item[1])

In [ ]:
rankingmeasure.get_NDCG(train, test, bpr_mf, True, K=5)

In [ ]:
imf = rankingmethod.iMF(user_item, K=20, lmbd=0.01, alpha=40, maxiter=30, verbose=2)

In [ ]:
imf.fit(train)

In [ ]:
all_measures(train, test, imf, True)

In [ ]:
all_measures(train, test, imf, True, top)

In [ ]:
all_measures(train, test, bpr_mf, True, top)

In [ ]:
all_measures(train, test, poprec, True)

In [ ]:
all_measures(train, test, poprec, True, top)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
a = np.array([False,False, True, True])
b = np.array([0,2,1,3])
print(roc_auc_score(a, b, average=None))

In [ ]:
(a == True).any()

In [ ]:
import random

In [ ]:
random.sample(range(10), 3)